In [43]:
from IPython.display import clear_output
from functions import *
import numpy as np
import pandas as pd
from transformers import *
from transformers.tokenization_utils import TextInputPair
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import pickle
import scipy as sc
import math as mt
from joblib import dump, load
import random
from sklearn.neighbors import KNeighborsTransformer

In [2]:
# BERT
casing = "bert-base-uncased" 
tokenizer = BertTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)

config = BertConfig(dropout=0.2, attention_dropout=0.2 ) #hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2
config.output_hidden_states = False # if true outputs all layers

model = TFBertModel.from_pretrained(casing, config = config)
model.trainable = False
emb_len = 768
clear_output()

# BERT
n_cluster = 27 # Number of clusters to use
n_pc = 12 # Number of main principal components to drop for local method
n_pc_global = 15 # Number of main principal components to drop for global method

In [104]:
# GPT-2
casing = "gpt2" 
tokenizer = GPT2Tokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = GPT2Config()
config.output_hidden_states = True

model = TFGPT2Model.from_pretrained(casing, config=config)
model.trainable = False

emb_len = 768
clear_output()

# GPT2
n_cluster = 10
n_pc = 30
n_pc_global = 30

In [146]:
# RoBERTa
casing = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = RobertaConfig.from_pretrained(casing)
config.output_hidden_states = True

model = TFRobertaModel.from_pretrained(casing, config=config)
model.trainable = False
emb_len = 768
clear_output()

# RoBERTa
n_cluster = 27
n_pc = 12
n_pc_global = 25

In [147]:
with open('data.150k.pickle', 'rb') as f:
    x = pickle.load(f)

In [47]:
random.seed(1)
data_ = random.sample(x,200)

In [4]:
data_ = x[:200]

In [148]:
ids = []
for group in data_:
    gids = []
    for i in range(6):
        gids.append(tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(group[i])))
    ids.append(gids)

In [150]:
reps = []
for i in range(200):
    groupoutput = model(np.asarray(ids[i], dtype="int32"))[0]
    if casing == "gpt2":
        reps.append(np.asarray(groupoutput))
    else:
        reps.append(np.asarray(groupoutput)[:,1:groupoutput.shape[1]-1,:])

In [120]:
reps[0].shape

(6, 15, 768)

In [114]:
len(data_[0][0])

15

In [192]:
# Get all comma CWR indices, and all CWRs
token = "," # "." , "the" , "of"
tokenpositions = [] # list of tuple (group,sent,index,fullidx)
finalreps = []
tokenreps = []
cnt = 0
kix = 0
for i in range(200):
    for j in range(6):
        first_added = False
        for l in range(len(data_[i][j])):
            finalreps.append(reps[i][j][l])
            if data_[i][j][l] == token and not first_added:
                tokenreps.append(reps[i][j][l])
                tokenpositions.append( (i,j,l,cnt,kix) )
                first_added = True
                kix +=1
            cnt+=1

In [193]:
knn = KNeighborsTransformer(n_neighbors=6)
knn.fit_transform(tokenreps)

<666x666 sparse matrix of type '<class 'numpy.float32'>'
	with 4662 stored elements in Compressed Sparse Row format>

In [194]:
fullixs = set([pos[4] for pos in tokenpositions])
trueneighbors = []
for pos in tokenpositions:
    group,sent,index,fullidx,tokenidx = pos
    scores, neighs = knn.kneighbors([tokenreps[tokenidx]], 6)
    truegroupixs = set(map(lambda y: y[4], filter(lambda x: x[0] == group,tokenpositions)))
    trueneighbors.append(len(set(neighs[0]).intersection(truegroupixs))/6)

In [195]:
np.mean(trueneighbors)

0.6869369369369369

In [196]:
# Repeat with method..

iso_tokenreps = cluster_based(np.array(tokenreps),1,n_pc,emb_len)

In [197]:
knn = KNeighborsTransformer(n_neighbors=6)
knn.fit_transform(iso_tokenreps)

<666x666 sparse matrix of type '<class 'numpy.float64'>'
	with 4662 stored elements in Compressed Sparse Row format>

In [198]:
fullixs = set([pos[4] for pos in tokenpositions])
trueneighbors = []
for pos in tokenpositions:
    group,sent,index,fullidx,tokenidx = pos
    scores, neighs = knn.kneighbors([iso_tokenreps[tokenidx]], 6)
    truegroupixs = set(map(lambda y: y[4], filter(lambda x: x[0] == group,tokenpositions)))
    trueneighbors.append(len(set(neighs[0]).intersection(truegroupixs))/6)

In [199]:
np.mean(trueneighbors)

0.7064564564564565